In [58]:
from mixture_of_products_validation import track_log_likelihood
import pickle

with open('amewoo_mop_params_48_obs1.0_ent0.0001_dist0.01_pow0.4.pkl', 'rb') as f:
    params = pickle.load(f)

In [59]:
print(params.keys())

dict_keys(['MixtureOfProductsModel', 'MixtureOfProductsModel/Product0', 'MixtureOfProductsModel/Product1', 'MixtureOfProductsModel/Product2', 'MixtureOfProductsModel/Product3', 'MixtureOfProductsModel/Product4', 'MixtureOfProductsModel/Product5', 'MixtureOfProductsModel/Product6', 'MixtureOfProductsModel/Product7', 'MixtureOfProductsModel/Product8', 'MixtureOfProductsModel/Product9'])


In [19]:
import pandas as pd
amewoo_track_df = pd.concat([pd.read_csv(
            f"/Users/jacobepstein/Documents/work/birdflow_black_box_old/tracks/amewoo-Blomberg-track-data-res-48km.csv"),
                                     pd.read_csv(
                                         f"/Users/jacobepstein/Documents/work/birdflow_black_box_old/tracks/amewoo-track-data-res-48km.csv")],
                                    axis=0, ignore_index=True)
print(max(list(amewoo_track_df['st_week.2'])))
print(min(list(amewoo_track_df['st_week.1'])))

52
1


In [4]:
%%time
tll = track_log_likelihood(params, amewoo_track_df)
print(tll)

TypeError: track_log_likelihood() missing 1 required positional argument: 'conversion_dict'

In [5]:
# deep dive into update hdf preprocessing
import os
import h5py
import numpy as np
from mixture_of_products_model_training import Datatuple, mask_input

hdf_src = "/Users/jacobepstein/Documents/work/BirdFlowModels/amewoo_2021_48km.hdf5"
file = h5py.File(hdf_src, 'r')

true_densities = np.asarray(file['distr']).T

weeks = true_densities.shape[0]
total_cells = true_densities.shape[1]

distance_vector = np.asarray(file['distances'])**0.4
distance_vector *= 1 / (100**0.4)
masks = np.asarray(file['geom']['dynamic_mask']).T.astype(bool)

dtuple = Datatuple(weeks, total_cells, distance_vector, masks)
distance_matrices, masked_densities = mask_input(true_densities, dtuple)
cells = [d.shape[0] for d in masked_densities]
masks

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [10]:
# deep dive into st_densities
len(masked_densities)

53

In [6]:
print(len(params['MixtureOfProductsModel/Product1'].keys()))

53


In [53]:
from mixture_of_products_validation import to_dynamic_conversion_dict
conversion_dict = to_dynamic_conversion_dict(masks)
# conversion_dict = to_dynamic_conversion_dict(masks)
print(conversion_dict)

{0: {1254: 0, 1303: 1, 1304: 2, 1306: 3, 1307: 4, 1354: 5, 1355: 6, 1356: 7, 1358: 8, 1359: 9, 1406: 10, 1408: 11, 1410: 12, 1458: 13, 1498: 14, 1499: 15, 1500: 16, 1501: 17, 1542: 18, 1543: 19, 1544: 20, 1584: 21, 1585: 22, 1586: 23, 1624: 24, 1625: 25, 1626: 26, 1627: 27, 1628: 28, 1629: 29, 1665: 30, 1666: 31, 1667: 32, 1668: 33, 1669: 34, 1670: 35, 1671: 36, 1672: 37, 1708: 38, 1709: 39, 1710: 40, 1711: 41, 1712: 42, 1713: 43, 1714: 44, 1750: 45, 1751: 46, 1752: 47, 1753: 48, 1754: 49, 1755: 50, 1756: 51, 1757: 52, 1792: 53, 1793: 54, 1794: 55, 1795: 56, 1796: 57, 1797: 58, 1798: 59, 1799: 60, 1816: 61, 1817: 62, 1818: 63, 1819: 64, 1820: 65, 1832: 66, 1833: 67, 1834: 68, 1835: 69, 1836: 70, 1837: 71, 1838: 72, 1839: 73, 1840: 74, 1841: 75, 1858: 76, 1859: 77, 1860: 78, 1861: 79, 1862: 80, 1863: 81, 1864: 82, 1874: 83, 1875: 84, 1876: 85, 1877: 86, 1878: 87, 1879: 88, 1880: 89, 1881: 90, 1899: 91, 1900: 92, 1901: 93, 1902: 94, 1903: 95, 1904: 96, 1905: 97, 1906: 98, 1907: 99, 1908:

In [54]:
%%time

from mixture_of_products_validation import track_log_likelihood
tll = track_log_likelihood(params, amewoo_track_df, masks)
print(tll)

row 0 of 10440
2379
2379
2379
2379
'week_1412'
2379
2379
2379
2379
2379
2379
2379
2379
2379
'week_1013'
'week_1107'
'week_1019'
'week_396'
'week_422'
'week_127'
row 20 of 10440
'week_79'
185
185
185
'week_116'
'week_123'
'week_62'
185
185
185
'week_764'
2379
2379
2379
2379
row 40 of 10440
'week_418'
'week_621'
'week_803'
'week_936'
'week_978'
'week_996'
'week_1018'
'week_1027'
'week_1050'
'week_1044'
'week_1024'
'week_966'
'week_956'
'week_1002'
'week_843'
'week_907'
'week_764'
'week_874'
'week_835'
'week_478'
row 60 of 10440
'week_603'
'week_808'
'week_709'
'week_697'
'week_599'
'week_532'
'week_558'
'week_585'
'week_594'
2379
2379
2379
2379
'week_1242'
'week_764'
2379
2379
2379
2379
2379
row 80 of 10440
'week_1105'
'week_1081'
'week_888'
'week_932'
'week_952'
'week_975'
'week_985'
'week_494'
'week_496'
'week_486'
'week_450'
'week_468'
'week_527'
'week_370'
'week_432'
'week_594'
'week_595'
'week_490'
'week_535'
'week_498'
row 100 of 10440
'week_485'
'week_432'
'week_425'
'week_460'
'w

'week_539'
'week_565'
'week_592'
'week_601'
'week_620'
'week_622'
'week_610'
row 1360 of 10440
'week_573'
'week_587'
'week_642'
'week_485'
'week_547'
'week_711'
'week_712'
'week_607'
'week_652'
'week_614'
'week_602'
'week_549'
'week_544'
'week_579'
'week_579'
'week_705'
'week_747'
'week_666'
'week_656'
'week_664'
row 1380 of 10440
'week_678'
'week_573'
1487
'week_405'
'week_505'
'week_630'
'week_835'
'week_737'
'week_773'
'week_392'
'week_443'
'week_469'
'week_496'
'week_621'
'week_593'
'week_607'
'week_662'
'week_505'
'week_405'
'week_505'
row 1400 of 10440
'week_630'
'week_441'
'week_644'
'week_826'
'week_663'
'week_711'
'week_736'
'week_763'
'week_772'
'week_790'
'week_789'
'week_776'
'week_737'
'week_745'
'week_797'
'week_639'
'week_703'
'week_869'
'week_872'
'week_761'
row 1420 of 10440
'week_808'
'week_772'
'week_760'
'week_707'
'week_701'
'week_730'
'week_730'
'week_857'
'week_907'
'week_405'
'week_505'
'week_630'
'week_835'
'week_872'
'week_819'
'week_438'
'week_489'
'week_515'

'week_821'
'week_905'
'week_949'
'week_969'
'week_992'
'week_1002'
'week_1024'
'week_1021'
'week_1005'
'week_950'
'week_942'
'week_989'
'week_829'
'week_893'
'week_1060'
row 2120 of 10440
'week_1057'
'week_942'
'week_987'
'week_947'
'week_935'
'week_873'
'week_863'
'week_881'
'week_888'
'week_1014'
'week_1095'
'week_1019'
'week_1018'
'week_1028'
'week_1048'
'week_945'
'week_754'
'week_749'
'week_875'
'week_1080'
row 2140 of 10440
'week_1284'
'week_1263'
'week_863'
'week_608'
'week_634'
'week_660'
'week_669'
'week_632'
'week_634'
'week_622'
'week_585'
'week_599'
'week_706'
'week_549'
'week_612'
'week_776'
'week_724'
'week_619'
'week_664'
'week_626'
row 2160 of 10440
'week_667'
'week_614'
'week_610'
'week_644'
'week_644'
'week_770'
'week_813'
'week_732'
'week_722'
'week_749'
'week_875'
'week_1080'
'week_1284'
'week_1132'
'week_1255'
'week_1288'
'week_1291'
'week_545'
'week_573'
'week_562'
row 2180 of 10440
'week_749'
'week_875'
'week_1080'
'week_1284'
cannot convert float NaN to integer


'week_61'
'week_61'
'week_120'
'week_188'
'week_285'
'week_444'
'week_423'
'week_505'
'week_707'
'week_915'
'week_1099'
'week_1223'
'week_1256'
'week_971'
'week_994'
'week_1004'
row 2980 of 10440
'week_1026'
'week_790'
'week_453'
'week_417'
'week_435'
'week_497'
'week_340'
'week_402'
'week_469'
'week_456'
1673
2316
2316
'week_432'
2316
2316
'week_614'
'week_691'
'week_709'
'week_835'
row 3000 of 10440
'week_1040'
'week_690'
'week_425'
'week_669'
'week_439'
1673
1673
1673
1673
'week_137'
'week_143'
'week_140'
'week_156'
'week_198'
'week_289'
'week_380'
'week_489'
'week_648'
'week_750'
'week_503'
row 3020 of 10440
'week_437'
'week_546'
'week_725'
'week_859'
'week_904'
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
c

'week_574'
1393
'week_1059'
'week_782'
1393
'week_1059'
'week_1167'
1866
1866
1866
'week_577'
'week_643'
'week_578'
'week_573'
'week_580'
'week_600'
'week_645'
'week_683'
1393
1393
row 4260 of 10440
1393
1912
1912
'week_508'
'week_446'
'week_409'
'week_346'
'week_355'
'week_396'
'week_439'
'week_537'
'week_628'
'week_993'
'week_1097'
'week_1270'
'week_725'
'week_910'
'week_1041'
'week_1080'
'week_1095'
row 4280 of 10440
'week_1115'
'week_1121'
'week_1144'
'week_1138'
'week_1110'
'week_1046'
1393
1393
1393
'week_1059'
1744
1744
1744
1744
'week_634'
'week_602'
'week_538'
'week_533'
'week_539'
'week_603'
row 4300 of 10440
'week_828'
'week_935'
'week_1038'
'week_864'
'week_659'
'week_864'
1393
1393
1393
'week_1059'
'week_782'
1948
1948
'week_1289'
'week_890'
'week_721'
'week_574'
'week_541'
'week_475'
'week_466'
row 4320 of 10440
'week_470'
1393
1393
1393
'week_1059'
'week_782'
1827
1827
'week_544'
'week_396'
'week_359'
'week_456'
1393
1393
1393
'week_1059'
'week_782'
'week_498'
1393
1393


'week_252'
'week_122'
'week_144'
'week_115'
'week_114'
'week_120'
'week_399'
row 5120 of 10440
'week_413'
'week_427'
'week_323'
'week_86'
1606
1606
1606
1647
'week_630'
'week_533'
'week_528'
'week_534'
'week_735'
'week_440'
'week_549'
'week_707'
'week_928'
'week_810'
'week_995'
1481
row 5140 of 10440
1481
'week_396'
'week_345'
'week_326'
'week_460'
'week_399'
'week_413'
'week_427'
'week_323'
2271
2271
2269
2269
'week_481'
'week_444'
'week_380'
'week_371'
'week_375'
'week_390'
'week_432'
row 5160 of 10440
'week_293'
'week_355'
'week_399'
'week_413'
'week_427'
'week_323'
'week_86'
'week_730'
'week_325'
'week_175'
'week_64'
'week_86'
'week_86'
'week_152'
'week_427'
'week_323'
row 5180 of 10440
'week_86'
1825
1825
1825
1825
1825
1825
1825
'week_574'
'week_594'
'week_639'
'week_860'
2484
'week_778'
'week_983'
'week_1189'
'week_1357'
'week_1470'
'week_1242'
'week_730'
row 5200 of 10440
'week_314'
'week_324'
'week_344'
'week_346'
'week_510'
'week_500'
'week_510'
'week_524'
'week_784'
'week_11

'week_93'
'week_92'
'week_98'
'week_132'
'week_211'
'week_566'
'week_668'
'week_794'
'week_441'
'week_145'
'week_56'
'week_128'
row 6000 of 10440
'week_128'
'week_210'
'week_333'
'week_533'
'week_145'
'week_56'
1715
'week_77'
1715
1715
1715
1715
'week_89'
'week_162'
'week_287'
'week_487'
row 6020 of 10440
1409
'week_145'
'week_56'
'week_1076'
'week_145'
'week_56'
'week_145'
'week_56'
'week_54'
'week_54'
'week_57'
'week_57'
'week_79'
'week_146'
'week_221'
row 6040 of 10440
'week_324'
'week_442'
'week_543'
'week_669'
'week_874'
'week_1081'
'week_875'
'week_1007'
'week_1047'
'week_1063'
'week_1083'
'week_145'
'week_56'
'week_222'
'week_712'
1784
1784
1784
1784
row 6060 of 10440
'week_276'
'week_366'
'week_351'
'week_468'
'week_570'
'week_696'
'week_901'
'week_1108'
'week_875'
'week_1007'
'week_1047'
'week_1063'
'week_1083'
'week_991'
'week_1055'
'week_1141'
'week_1303'
'week_910'
'week_966'
'week_1055'
row 6080 of 10440
'week_1141'
'week_1303'
'week_1192'
'week_953'
'week_1084'
'week_1123

'week_570'
'week_240'
'week_204'
'week_224'
'week_289'
'week_143'
'week_191'
'week_354'
'week_361'
'week_257'
'week_306'
row 7340 of 10440
'week_270'
'week_256'
'week_205'
'week_200'
'week_235'
'week_234'
'week_307'
'week_300'
'week_776'
'week_1199'
1838
1838
1838
'week_385'
'week_222'
'week_120'
'week_94'
'week_75'
'week_73'
'week_79'
row 7360 of 10440
'week_75'
'week_258'
'week_366'
'week_525'
'week_627'
'week_753'
'week_742'
'week_907'
'week_1091'
'week_492'
'week_709'
'week_734'
'week_539'
'week_603'
'week_624'
'week_626'
'week_614'
'week_577'
'week_591'
'week_645'
row 7380 of 10440
'week_488'
'week_550'
'week_715'
'week_656'
'week_618'
'week_606'
'week_553'
'week_548'
'week_583'
'week_583'
'week_709'
'week_750'
'week_669'
'week_659'
'week_667'
'week_681'
'week_577'
1838
1838
'week_122'
row 7400 of 10440
'week_96'
'week_1289'
'week_120'
'week_94'
'week_75'
'week_73'
'week_79'
'week_75'
'week_102'
'week_177'
'week_258'
'week_366'
'week_525'
'week_627'
'week_753'
'week_958'
'week_689

'week_140'
'week_137'
'week_153'
'week_195'
'week_286'
'week_377'
'week_486'
'week_645'
'week_747'
'week_873'
'week_1078'
'week_1282'
'week_1434'
row 8380 of 10440
'week_1536'
'week_811'
'week_834'
'week_860'
'week_868'
'week_953'
'week_948'
'week_956'
'week_972'
'week_868'
'week_610'
'week_369'
'week_91'
'week_551'
'week_402'
'week_365'
'week_304'
'week_295'
'week_298'
'week_311'
row 8400 of 10440
'week_351'
'week_395'
'week_493'
'week_949'
'week_871'
'week_1534'
'week_956'
'week_972'
'week_868'
'week_610'
'week_369'
1154
1154
1154
1532
'week_90'
'week_88'
'week_93'
'week_264'
'week_303'
row 8420 of 10440
'week_347'
'week_445'
'week_802'
'week_905'
'week_535'
'week_738'
'week_946'
'week_1129'
'week_813'
'week_858'
'week_953'
'week_948'
'week_956'
'week_972'
'week_868'
'week_610'
'week_369'
'week_91'
'week_119'
'week_119'
row 8440 of 10440
'week_247'
'week_455'
'week_446'
'week_450'
'week_953'
'week_948'
'week_956'
'week_972'
'week_868'
'week_610'
'week_369'
1154
1154
'week_574'
'week_

'week_768'
'week_789'
2405
2405
2405
2405
2405
2405
2405
2405
2405
'week_1350'
'week_334'
row 9960 of 10440
'week_346'
'week_366'
'week_366'
'week_359'
'week_323'
'week_343'
'week_407'
'week_250'
'week_311'
'week_473'
'week_477'
'week_373'
'week_367'
'week_409'
'week_451'
'week_549'
'week_640'
'week_751'
'week_903'
2246
row 9980 of 10440
2246
'week_1055'
1936
1936
'week_1315'
'week_1383'
'week_1382'
'week_1390'
'week_1374'
1724
1724
1724
1724
'week_1202'
'week_1392'
'week_1323'
'week_1047'
'week_738'
'week_332'
'week_182'
row 10000 of 10440
'week_68'
1724
'week_448'
'week_489'
'week_587'
'week_678'
'week_448'
'week_489'
'week_587'
'week_678'
'week_789'
'week_941'
2289
2289
'week_1209'
'week_1055'
'week_548'
'week_684'
'week_395'
'week_421'
row 10020 of 10440
'week_448'
'week_457'
'week_477'
'week_479'
'week_469'
'week_433'
'week_527'
'week_540'
'week_435'
'week_641'
'week_846'
'week_1053'
'week_1235'
'week_402'
'week_395'
'week_421'
'week_448'
'week_457'
'week_477'
'week_479'
row 10040

In [4]:
file['geom']['dynamic_mask']
total_distances = len(distance_vector)
computed = 0.5 * (total_cells) * (total_cells - 1)
print(computed, total_distances)
print(distance_vector)

3465028.0 3465028
[0.98026361 1.15280473 1.41400373 ... 0.85540024 0.74849188 0.74320277]


In [5]:
import jax.numpy as jnp
distance_matrix = jnp.zeros((dtuple.cells, dtuple.cells))
distance_matrix = distance_matrix.at[jnp.triu_indices(dtuple.cells, k=1)].set(dtuple.distances)
print(distance_matrix)
distance_matrix = distance_matrix + distance_matrix.T

[[0.         0.9802636  1.1528047  ... 4.090942   4.1029058  4.1122456 ]
 [0.         0.         0.7427801  ... 4.072277   4.0851455  4.0939765 ]
 [0.         0.         0.         ... 4.0633235  4.0766497  4.085215  ]
 ...
 [0.         0.         0.         ... 0.         0.85540026 0.7484919 ]
 [0.         0.         0.         ... 0.         0.         0.74320275]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [6]:
distance_matrices = []
print(len(list(filter(lambda b: b, dtuple.masks[0])))) # size of first dimension
print(len(list(filter(lambda b: b, dtuple.masks[1])))) # size of second dimension
for i in range(0, dtuple.weeks - 1):
    distance_matrices.append(distance_matrix[dtuple.masks[i], :][:, dtuple.masks[i + 1]])
distance_matrices[0].shape

675
702


(675, 702)

In [7]:
test_matrix = np.random.rand(2, 3)
print(f"test matrix before: {test_matrix}")
test_matrix[[True, False], :][:, [True, False, True]] # masking example

test matrix before: [[0.54044845 0.27857001 0.28957231]
 [0.18794457 0.14794973 0.48391876]]


array([[0.54044845, 0.28957231]])

In [8]:
masked_densities = []
for density, mask in zip(true_densities, dtuple.masks):
    masked_densities.append(density[mask])
print(masked_densities[1]) # length 702

# exploration: when we filter the cells that normally get filtered, are all the probabilities basically zero (any normalization that needs to occur here?)
print(len(true_densities[1][dtuple.masks[1]])) # this is the element we want
inverse_mask = list(map(lambda b: not b, list(dtuple.masks[1])))
cells_that_normally_get_masked = true_densities[1][inverse_mask]
print(cells_that_normally_get_masked, sum(cells_that_normally_get_masked)) # yup! everything is zero

[6.29806756e-05 3.94161304e-06 1.82066513e-05 1.06478402e-05
 2.60056834e-05 1.38480471e-05 4.81991837e-05 8.15641180e-05
 4.84213254e-05 1.06478402e-05 1.38480471e-05 1.39526162e-04
 1.40541641e-05 2.36010942e-05 6.26965526e-05 6.41858193e-05
 5.73918919e-05 6.55580338e-04 7.63642378e-05 1.89003569e-04
 1.25182836e-03 1.26157743e-04 2.47612980e-06 5.85409604e-06
 1.70902658e-04 2.20625099e-04 5.09033055e-04 6.71475468e-04
 2.15757786e-05 7.17691769e-07 1.53127549e-04 2.18355437e-04
 7.15538784e-05 3.87141568e-04 1.79536158e-04 1.26350317e-03
 1.23220791e-03 5.21501024e-05 1.50995840e-03 8.62644345e-04
 5.00257067e-04 1.68626480e-03 6.39369675e-04 2.27032436e-04
 4.76459594e-05 6.95347509e-04 1.18722121e-03 2.73938223e-03
 1.71944743e-03 8.77525140e-04 4.47785200e-04 1.74892395e-04
 1.39439858e-06 1.96917289e-07 3.88223735e-04 1.99288330e-03
 4.74047411e-03 8.78620124e-03 4.72131979e-03 1.24196435e-03
 6.00007465e-04 8.46848788e-04 4.21432761e-05 1.20843583e-04
 4.43868721e-05 3.816879

In [126]:
# final question: in the track data, are we using the same original ~2600 cell grid every week?
# or are we using this dynamic grid that masks out cells with zero density each week?
print(amewoo_track_df)
amewoo_track_df['cell.1'][0], amewoo_track_df['cell.2'][0], amewoo_track_df['st_week.1'][0], cells[5:7]

       Unnamed: 0            indiv  year      lon.1      lat.1  cell.1  row.1  \
0               1  AL-2020-01_2020  2020 -87.174667  32.083878  2379.0     51   
1               2  AL-2020-01_2020  2020 -87.175385  32.083366  2379.0     51   
2               3  AL-2020-01_2020  2020 -87.175514  32.083622  2379.0     51   
3               4  AL-2020-01_2020  2020 -87.175514  32.083416  2379.0     51   
4               5  AL-2020-01_2020  2020 -82.970482  38.171097  1785.0     37   
...           ...              ...   ...        ...        ...     ...    ...   
10435        1530  Woodcock 8_2014  2014 -86.657000  36.199000  1947.0     41   
10436        1531  Woodcock 9_2014  2014 -93.701000  45.528000   915.0     19   
10437        1532  Woodcock 9_2014  2014 -93.705000  45.526000   915.0     19   
10438        1533  Woodcock 9_2014  2014 -93.708000  45.524000   915.0     19   
10439        1534  Woodcock 9_2014  2014 -91.041000  38.370000  1729.0     36   

       col.1            x.1

(2379.0, 2379.0, 6, [954, 1055])

In [9]:
"""
bugs:
st_week.1 / st_week.2 do not directly translate to the correct weeks of the mixture of products model due to zero indexing
using the overall grid each week in the track data instead of the weekly filtered grids

Questions:
- are the weeks zero indexed in the track data? (I don't think so)
"""

'\npotential bugs:\nst_week.1 / st_week.2 do not directly translate to the correct weeks of the mixture of products model due to zero indexing\n\n\n'

In [27]:
# why does this not error out?
from mixture_of_products_model import get_forecast_prob
import math

week_1 = int(amewoo_track_df['st_week.1'][0])
week_2 = int(amewoo_track_df['st_week.2'][0])
cell_1 = int(amewoo_track_df['cell.1'][0])
cell_2 = int(amewoo_track_df['cell.2'][0])
print(week_1, week_2, cell_1, cell_2)
print(get_forecast_prob(params, [(week_2, cell_2)], [(week_1, cell_1)]))

transition_prob = get_forecast_prob(params, [(int(amewoo_track_df['st_week.2'][0]), int(amewoo_track_df['cell.2'][0]))], [(int(amewoo_track_df['st_week.1'][0]), int(amewoo_track_df['cell.1'][0]))])
ll_of_transition = math.log(transition_prob)
print(ll_of_transition)

6 7 2379 2379
8.4951424e-08
-16.281186230012175


In [58]:
from jax.nn import softmax
conditions = [(week_1, cell_1)]
observations = [(week_2, cell_2)]
n_products = len(params.keys()) - 1
# compute weights pi for each of the corresponding conditionals of the product distributions
weights = softmax(params['MixtureOfProductsModel']['weights'])
pi = jnp.zeros(n_products)
for r in range(n_products):
    likelihood_r = 1
    for (t, obs) in conditions:
        likelihood_r *= softmax(params[f'MixtureOfProductsModel/Product{r}'][f'week_{t}'])[obs]
    pi = pi.at[r].set(weights[r] * likelihood_r)
pi /= pi.sum()  # normalize the weights

# compute the final conditional, summing over the conditionals for each product k weighted by pi[k]
conditional_prob = 0
for k in range(n_products):
    prod_k_conditional_prob = jnp.asarray(1)
    for tstep, cell in observations:
        prod_k_conditional_prob *= softmax(params[f'MixtureOfProductsModel/Product{k}'][f'week_{tstep}'])[cell]
    conditional_prob += pi[k] * prod_k_conditional_prob

# KEY NOTE: if we index with something larger than the jax array size, we just get the last element of the array
print(params['MixtureOfProductsModel/Product6']['week_6'][0])
print(len(params['MixtureOfProductsModel/Product6']['week_6']))
print(params['MixtureOfProductsModel/Product6']['week_6'][1054], params['MixtureOfProductsModel/Product6']['week_6'][1080])



-4.363604
1055
-3.8879373 -3.8879373


In [65]:
print(max(list(amewoo_track_df['cell.1'])), total_cells)
print(max(amewoo_track_df['st_week.1']))

2630.0 2633
51


In [31]:
d = {2: 3}
d[2]
d

{2: 3}

In [33]:
list(enumerate([5, 32, 2]))

[(0, 5), (1, 32), (2, 2)]

In [52]:
from mixture_of_products_validation import to_dynamic_conversion_dict
conversion_dict = to_dynamic_conversion_dict(masks)
week_1 = int(amewoo_track_df['st_week.1'][0])
week_2 = int(amewoo_track_df['st_week.2'][0])
print(week_1, week_2)
print(conversion_dict[7])
print(masks[7][1442])
cell_1 = conversion_dict[int(amewoo_track_df['st_week.1'][0])][int(amewoo_track_df['cell.1'][0])]  # convert to indices in dynamic mask
cell_2 = conversion_dict[int(amewoo_track_df['st_week.2'][0])][int(amewoo_track_df['cell.2'][0])]


6 7
{1105: 0, 1141: 1, 1142: 2, 1158: 3, 1172: 4, 1182: 5, 1183: 6, 1189: 7, 1190: 8, 1191: 9, 1194: 10, 1195: 11, 1196: 12, 1197: 13, 1205: 14, 1206: 15, 1219: 16, 1222: 17, 1224: 18, 1227: 19, 1229: 20, 1235: 21, 1237: 22, 1238: 23, 1241: 24, 1244: 25, 1247: 26, 1250: 27, 1251: 28, 1252: 29, 1253: 30, 1269: 31, 1270: 32, 1271: 33, 1272: 34, 1274: 35, 1276: 36, 1277: 37, 1278: 38, 1280: 39, 1282: 40, 1285: 41, 1286: 42, 1287: 43, 1289: 44, 1290: 45, 1293: 46, 1294: 47, 1297: 48, 1298: 49, 1300: 50, 1301: 51, 1302: 52, 1303: 53, 1304: 54, 1306: 55, 1317: 56, 1321: 57, 1322: 58, 1323: 59, 1324: 60, 1325: 61, 1326: 62, 1327: 63, 1328: 64, 1329: 65, 1330: 66, 1331: 67, 1332: 68, 1333: 69, 1334: 70, 1335: 71, 1337: 72, 1339: 73, 1340: 74, 1341: 75, 1342: 76, 1343: 77, 1344: 78, 1345: 79, 1346: 80, 1347: 81, 1348: 82, 1349: 83, 1350: 84, 1351: 85, 1352: 86, 1353: 87, 1354: 88, 1355: 89, 1356: 90, 1358: 91, 1359: 92, 1369: 93, 1370: 94, 1372: 95, 1373: 96, 1374: 97, 1375: 98, 1376: 99, 1377:

KeyError: 2379

In [62]:
with open('../st_and_track_data_processing/amewoo_params_48_obs1.0_ent6e-06_dist0.025035_pow0.5.pkl', 'rb') as f:
    markov_params = pickle.load(f)
print(markov_params.keys()) # an initial distribution and 52 weekly distributions
print(len(markov_params['Flow_Model/Initial_Params']['z0']), cells[0])


dict_keys(['Flow_Model/Initial_Params', 'Flow_Model/Week_1', 'Flow_Model/Week_10', 'Flow_Model/Week_11', 'Flow_Model/Week_12', 'Flow_Model/Week_13', 'Flow_Model/Week_14', 'Flow_Model/Week_15', 'Flow_Model/Week_16', 'Flow_Model/Week_17', 'Flow_Model/Week_18', 'Flow_Model/Week_19', 'Flow_Model/Week_2', 'Flow_Model/Week_20', 'Flow_Model/Week_21', 'Flow_Model/Week_22', 'Flow_Model/Week_23', 'Flow_Model/Week_24', 'Flow_Model/Week_25', 'Flow_Model/Week_26', 'Flow_Model/Week_27', 'Flow_Model/Week_28', 'Flow_Model/Week_29', 'Flow_Model/Week_3', 'Flow_Model/Week_30', 'Flow_Model/Week_31', 'Flow_Model/Week_32', 'Flow_Model/Week_33', 'Flow_Model/Week_34', 'Flow_Model/Week_35', 'Flow_Model/Week_36', 'Flow_Model/Week_37', 'Flow_Model/Week_38', 'Flow_Model/Week_39', 'Flow_Model/Week_4', 'Flow_Model/Week_40', 'Flow_Model/Week_41', 'Flow_Model/Week_42', 'Flow_Model/Week_43', 'Flow_Model/Week_44', 'Flow_Model/Week_45', 'Flow_Model/Week_46', 'Flow_Model/Week_47', 'Flow_Model/Week_48', 'Flow_Model/Week_4

In [48]:
from jax.nn import softmax
arr = sorted(enumerate(softmax(params['MixtureOfProductsModel/Product5']['week_20'])), key=lambda tup: -tup[1])
arr

[(167, Array(0.9759278, dtype=float32)),
 (324, Array(0.02358244, dtype=float32)),
 (431, Array(4.0464485e-05, dtype=float32)),
 (599, Array(1.4560034e-05, dtype=float32)),
 (273, Array(1.0930825e-05, dtype=float32)),
 (606, Array(9.932301e-06, dtype=float32)),
 (651, Array(9.4562865e-06, dtype=float32)),
 (425, Array(9.343497e-06, dtype=float32)),
 (540, Array(8.804365e-06, dtype=float32)),
 (595, Array(8.625424e-06, dtype=float32)),
 (760, Array(8.090968e-06, dtype=float32)),
 (600, Array(7.745655e-06, dtype=float32)),
 (653, Array(7.5809394e-06, dtype=float32)),
 (542, Array(7.1095005e-06, dtype=float32)),
 (432, Array(6.6345992e-06, dtype=float32)),
 (539, Array(6.3562843e-06, dtype=float32)),
 (485, Array(6.320728e-06, dtype=float32)),
 (656, Array(5.7457983e-06, dtype=float32)),
 (377, Array(5.3068843e-06, dtype=float32)),
 (429, Array(5.165188e-06, dtype=float32)),
 (765, Array(5.138588e-06, dtype=float32)),
 (373, Array(4.7634794e-06, dtype=float32)),
 (322, Array(4.4823155e-06